In [753]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import warnings
from sklearn import linear_model, tree, discriminant_analysis, naive_bayes, ensemble, gaussian_process
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, FunctionTransformer, LabelEncoder, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold, train_test_split
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 100)
from pycaret.classification import *
from function.data_engineering import *
from sklearn.svm import SVC

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [754]:
def calculate_loss_odds(win_odds, draw_odds):
    loss_odds = 1 / (1 / win_odds + 1 / draw_odds)
    return loss_odds

In [755]:
data = pd.read_csv('penyisihan-satria-data-itb-2023/train.csv')
test = pd.read_csv('penyisihan-satria-data-itb-2023/test.csv')
data.index = data.id
test.index = test.id
data = data.iloc[:,1:]
test = test.iloc[:,1:]
or_cols = [c for c in data.columns if c != 'target']
int_cols = [c for c in data.columns if data[c].dtype=='int64' and c != 'target']
float_cols = [c for c in data.columns if data[c].dtype=='float64' and c != 'target']
h_columns = [c for c in data.columns if c[-1]=='h']
d_columns = [c for c in data.columns if c[-1]=='d']

In [756]:
data['target'] = (data['target']).astype('category')


In [757]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 600 entries, 1 to 600
Data columns (total 36 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   hs           600 non-null    int64   
 1   hst          600 non-null    int64   
 2   hf           600 non-null    int64   
 3   hc           600 non-null    int64   
 4   hy           600 non-null    int64   
 5   hr           600 non-null    int64   
 6   b365h        600 non-null    float64 
 7   b365d        600 non-null    float64 
 8   bwh          600 non-null    float64 
 9   bwd          600 non-null    float64 
 10  iwh          600 non-null    float64 
 11  iwd          600 non-null    float64 
 12  psh          600 non-null    float64 
 13  psd          600 non-null    float64 
 14  whh          600 non-null    float64 
 15  whd          600 non-null    float64 
 16  vch          600 non-null    float64 
 17  vcd          600 non-null    float64 
 18  bb_mx_h      600 non-null    f

In [759]:
X = [c for c in data.columns if c != 'target']
scale = StandardScaler()
scale.fit(data[X])
data[X] = scale.transform(data[X])
test[X] = scale.transform(test[X])

In [763]:
tmp_train = data.copy()
tmp_test = test.copy()

In [764]:
tmp_train

,hs,hst,hf,hc,hy,hr,b365h,b365d,bwh,bwd,iwh,iwd,psh,psd,whh,whd,vch,vcd,bb_mx_h,bb_av_h,bb_mx_d,bb_av_d,bb_ou,bb_mx_2_5,bb_av_2_5,bb_mx_2_5_1,bb_av_2_5_1,bb_ah,bb_a_hh,bb_mx_ahh,bb_av_ahh,bb_mx_aha,bb_av_aha,psch,pscd,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.524832,0.434034,-1.012809,1.429838,0.430123,-0.213399,-0.418164,-0.540222,-0.430541,-0.505686,-0.453979,-0.513666,-0.438482,-0.509535,-0.461495,-0.477347,-0.425416,-0.539942,-0.407310,-0.422638,-0.531913,-0.524689,0.940543,0.483659,0.507003,-0.620425,-0.630216,1.007366,-0.363559,1.265339,1.400863,-1.311788,-1.296572,-0.435985,-0.585915,0
2,1.181208,0.067245,-1.631319,0.749503,-0.386567,-0.213399,-0.673929,1.071365,-0.704861,1.005207,-0.764299,1.103441,-0.687074,0.894218,-0.698224,1.074059,-0.678438,0.839573,-0.655619,-0.698576,0.915718,0.940038,0.226207,-1.083331,-1.072879,0.896919,0.995728,-0.038343,-1.351714,0.549206,0.581379,-0.717649,-0.724723,-0.661560,0.370583,0
3,0.493960,0.434034,-0.703555,0.409335,-0.386567,-0.213399,-0.673929,1.071365,-0.696420,0.854118,-0.734745,0.892514,-0.698912,0.861188,-0.715133,1.074059,-0.678438,0.839573,-0.652072,-0.702757,0.915718,0.964249,-0.130962,-1.110822,-1.102688,1.033091,0.974049,1.530221,-1.104676,-0.729603,-0.948326,0.902732,0.876454,-0.703616,0.812835,0
4,2.040268,2.267979,0.533464,0.069167,-0.386567,-0.213399,-0.437251,-0.384435,-0.464304,-0.354596,-0.453979,-0.513666,-0.446374,-0.437971,-0.436131,-0.322206,-0.444584,-0.401991,-0.421499,-0.443543,-0.382158,-0.409690,1.297711,-0.286090,-0.357461,-0.036831,-0.023197,-0.561197,-0.116520,-0.371537,-0.402003,0.200567,0.247420,-0.390105,-0.498493,0
5,1.009396,0.434034,0.224210,-0.951336,1.246814,-0.213399,-0.208207,-0.701381,-0.181543,-0.747428,-0.158436,-0.794902,-0.205674,-0.740741,-0.186720,-0.725572,-0.214564,-0.705484,-0.212210,-0.201052,-0.731586,-0.754688,-0.488130,1.528319,1.520512,-1.067847,-1.085480,-0.038343,0.130519,0.702663,0.799908,-0.879687,-0.781908,-0.141590,-0.735047,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.322148,0.067245,-1.012809,-0.271000,-1.203257,-0.213399,-0.570860,-0.164185,-0.574031,-0.203507,-0.651008,0.048806,-0.576589,-0.234289,-0.588314,-0.167066,-0.571095,-0.264039,-0.556295,-0.577331,-0.182485,-0.197849,-0.130962,0.263731,0.298339,-0.484253,-0.478461,0.484512,-0.610598,0.191139,0.144320,-0.231534,-0.267244,-0.600387,-0.092239,1
597,1.181208,2.634768,-1.631319,0.409335,-0.386567,-0.213399,-0.719738,2.145757,-0.747064,1.911742,-0.788928,1.595604,-0.734425,1.923636,-0.748952,1.694621,-0.720608,1.667282,-0.691092,-0.744566,1.914084,1.878190,-1.202466,-1.248277,-1.251734,1.324887,1.559389,0.484512,-1.598753,-0.524994,-0.620532,0.470631,0.476160,-0.745673,2.684692,1
598,0.150336,1.167612,0.533464,-0.271000,1.246814,-0.213399,-0.437251,-0.540222,-0.434761,-0.566121,-0.429350,-0.548820,-0.446374,-0.526049,-0.436131,-0.663516,-0.463752,-0.539942,-0.432141,-0.443543,-0.531913,-0.561005,1.297711,1.225918,1.192612,-0.912222,-0.955405,-0.561197,-0.116520,-0.524994,-0.511267,0.416618,0.418975,-0.462748,-0.524206,1


In [732]:
data['target']

id
1      0
2      0
3      0
4      0
5      0
      ..
596    1
597    1
598    1
599    0
600    0
Name: target, Length: 600, dtype: category
Categories (2, int64): [0, 1]

In [765]:
for i in int_cols:
    for c in int_cols:
        tmp_train[f'{c}_binding_encoded'] = pd.cut(tmp_train[c], bins=5, labels=[1,2,3,4,5])

for i in int_cols:
    for c in int_cols:
        tmp_test[f'{c}_binding_encoded'] = pd.cut(test[c], bins=5, labels=[1,2,3,4,5])


In [766]:
tmp_train= tmp_train.loc[~((tmp_train['hst_binding_encoded']>=5) & (tmp_train['target']==0 ))]
tmp_train= tmp_train.loc[~((tmp_train['hst_binding_encoded']<=1) & (tmp_train['target']==1 ))]
tmp_train= tmp_train.loc[~((tmp_train['hs_binding_encoded']>=5) & (tmp_train['target']==0 ))]
tmp_train= tmp_train.loc[~((tmp_train['hs_binding_encoded']<=1) & (tmp_train['target']==1 ))]
tmp_train= tmp_train.loc[~((tmp_train['hy_binding_encoded']>=4) & (tmp_train['target']==1 ))]
tmp_train= tmp_train.loc[~((tmp_train['hy_binding_encoded']<=1) & (tmp_train['target']==0 ))]
# tmp_train= tmp_train.loc[~((tmp_train['hr_binding_encoded']>=5) & (tmp_train['target']==1 ))]
# tmp_train= tmp_train.loc[~((tmp_train['hr_binding_encoded']<=1) & (tmp_train['target']==0 ))]


In [767]:
#tmp_train['ratio_hs'] = data['hst']/data['hs']
tmp_train['h_max'] = tmp_train[h_columns[:6]].max(axis=1)
tmp_train['h_mean'] = tmp_train[h_columns[:6]].mean(axis=1)
tmp_train['h_min'] = tmp_train[h_columns[:6]].min(axis=1)
tmp_train['d_max'] = tmp_train[d_columns[:6]].max(axis=1)
tmp_train['d_mean'] = tmp_train[d_columns[:6]].mean(axis=1)
tmp_train['d_min'] = tmp_train[d_columns[:6]].min(axis=1)

tmp_test['h_max'] = tmp_test[h_columns[:6]].max(axis=1)
tmp_test['h_mean'] = tmp_test[h_columns[:6]].mean(axis=1)
tmp_test['h_min'] = tmp_test[h_columns[:6]].min(axis=1)
tmp_test['d_max'] = tmp_test[d_columns[:6]].max(axis=1)
tmp_test['d_mean'] = tmp_test[d_columns[:6]].mean(axis=1)
tmp_test['d_min'] = tmp_test[d_columns[:6]].min(axis=1)

In [736]:
# win = tmp_train['h_max']
# draw = tmp_train['d_max']
# tmp_train['loss_max'] = abs(1 / (1 / win + 1 / draw))

# win = tmp_train['h_min']
# draw = tmp_train['d_min']
# tmp_train['loss_min'] = 1 / (1 / win + 1 / draw)

# win = tmp_train['h_mean']
# draw = tmp_train['d_mean']
# tmp_train['loss_mean'] = abs(1 / (1 / win + 1 / draw))

In [737]:
tmp_train['target']

id
1      0
5      0
7      1
8      0
14     1
      ..
595    1
596    1
597    1
598    1
599    0
Name: target, Length: 365, dtype: category
Categories (2, int64): [0, 1]

In [738]:
tmp_train['target']

id
1      0
5      0
7      1
8      0
14     1
      ..
595    1
596    1
597    1
598    1
599    0
Name: target, Length: 365, dtype: category
Categories (2, int64): [0, 1]

In [739]:
tmp_train['target'] = (tmp_train['target']).astype('int')

In [768]:
abs(tmp_train.corr()['target']).sort_values(ascending=False)[1:]

KeyError: 'target'

In [769]:
#tmp = tmp_train.drop(or_cols,axis=1)
tmp = tmp_train.copy()

In [770]:
X = [i for i in tmp.columns if i != 'target']
cross_validation(data[or_cols], data['target'],discriminant_analysis.LinearDiscriminantAnalysis())

              precision    recall  f1-score   support

           0       0.67      0.63      0.65        60
           1       0.65      0.68      0.67        60

    accuracy                           0.66       120
   macro avg       0.66      0.66      0.66       120
weighted avg       0.66      0.66      0.66       120
 0.6581196581196581
              precision    recall  f1-score   support

           0       0.70      0.88      0.78        60
           1       0.84      0.62      0.71        60

    accuracy                           0.75       120
   macro avg       0.77      0.75      0.75       120
weighted avg       0.77      0.75      0.75       120
 0.745475113122172
              precision    recall  f1-score   support

           0       0.63      0.73      0.68        60
           1       0.68      0.57      0.62        60

    accuracy                           0.65       120
   macro avg       0.65      0.65      0.65       120
weighted avg       0.65      0.65    

[0.6581196581196581,
 0.745475113122172,
 0.6475524475524476,
 0.7245217391304348,
 0.7416487256059449]

In [743]:
np.mean([0.6581196581196581,
 0.745475113122172,
 0.6475524475524476,
 0.7245217391304348,
 0.7416487256059449])

0.7034635367061315

In [771]:
tmp.target.value_counts()

1    219
0    146
Name: target, dtype: int64

In [791]:
from imblearn.ensemble import BalanceCascade
from sklearn.linear_model import LogisticRegression
bc = BalanceCascade(random_state=0,
                    estimator=LogisticRegression(random_state=0),
                    n_max_subset=10)


bc.fit(tmp[X], tmp['target'])
X_resampled, y_resampled = bc.sample(tmp[X], tmp['target'])
colors = ['#ef8a62' if v == 0 else '#f7f7f7' if v == 1 else '#67a9cf' for v in y_resampled[0, :]]
plt.scatter(X_resampled[0, :, 0], X_resampled[0, :, 1], c=colors, linewidth=1, edgecolor='black')
sns.despine()

ImportError: cannot import name 'BalanceCascade' from 'imblearn.ensemble' (/opt/anaconda3/lib/python3.9/site-packages/imblearn/ensemble/__init__.py)

In [772]:
X_train, X_test, y_train, y_test = train_test_split(tmp[X], tmp['target'], test_size=0.2, random_state=123)

In [785]:
clf = discriminant_analysis.LinearDiscriminantAnalysis(solver='svd').fit(X_train, y_train)
#clf  = LogisticRegression(random_state=0).fit(X_train, y_train)
pred = clf.predict(X_test)

accuracy_score(pred,y_test)

0.821917808219178

In [774]:
### Data :


In [786]:
clf = discriminant_analysis.LinearDiscriminantAnalysis(solver='svd').fit(tmp[X], tmp['target'])
clf.predict(tmp_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [787]:
print(pred, np.array(y_test),sep='\n')
print(pred.sum(), np.array(y_test).sum(),sep='\n')

[0 1 1 0 0 0 1 0 1 1 1 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 1 0
 1 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 1 1 0 1 0 0 0 0 1 0 0 1 0 1 1 1 1]
[0 1 1 0 0 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 1 1 0 1 0 1 0 0 1 0 0 1 0 1 0
 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 1 0 1 0 1 1 0 1 1 0 1 1 1 1]
33
36


In [788]:
pred.sum(), len(pred)

(33, 73)

In [789]:
confusion_matrix(pred,y_test)

array([[32,  8],
       [ 5, 28]])